<a href="https://colab.research.google.com/github/hansjang/Py_Radar/blob/main/%EB%B9%85%EB%B6%84%EA%B8%B0Part3_P212.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


(P.212) 3. 실습 A to Z : Census Income
---



In [ ]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata

**(1) 문제**

In [40]:
import pandas as pd
import numpy as np

X_test=pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/census_X_test.csv")
X_train=pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/census_X_train.csv")
y_train=pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/census_y_train.csv")

**(2) 전처리**

In [ ]:
print(X_train.info())
print(X_test.info())
print(y_train.info())

12개의 컬럼이 있고 결측치는 없다.
age, education_num, capital_gain, capital_loss, hours_per_week 는 수치형 변수이고,
workclass, marital_status, occupation, relationship, race, sex, native_country 는 범주형 변수이다.  

In [ ]:
print(X_train.describe())

capital_gain, capital_loss의 경우 오른쪽으로 크게 치우쳐져 있는 것을 확인할 수 있다.

In [ ]:
print(X_train['capital_gain'].quantile([q/20 for q in range(14, 21)]))

In [ ]:
print(X_train['capital_loss'].quantile([q/20 for q in range(14, 21)]))

**[초매개변수 최적화?]** capital_gain 변수는 전체 10% 이내, capital_loss 변수는 전체 5% 이내에서만 0보다 큰 값을 갖는다. 이 경우 범주형 변수로 변환하여 사용할 수 있다.

In [41]:
X_train['capital_gain_yn'] = np.where(X_train['capital_gain']>0, 1, 0)
X_train['capital_loss_yn'] = np.where(X_train['capital_loss']>0, 1, 0)

X_test['capital_gain_yn'] = np.where(X_test['capital_gain']>0, 1, 0)
X_test['capital_loss_yn'] = np.where(X_test['capital_loss']>0, 1, 0)

변수 종류별로 나눠 준다.

In [42]:
COL_DEL = []
COL_NUM = ['age', 'education_num', 'hours_per_week', 'capital_gain', 'capital_loss']
COL_CAT = ['workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'capital_gain_yn', 'capital_loss_yn' ]
COL_Y = ['target']

# axis = 0 은 행을 기준으로, axis = 1은 열을 기준으로 한다는 뜻
X_train = X_train.drop(COL_DEL, axis = 1)
X_test = X_test.drop(COL_DEL, axis = 1)

In [43]:
print(X_train.info())
print(X_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30162 entries, 0 to 30161
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              30162 non-null  int64 
 1   workclass        30162 non-null  object
 2   education_num    30162 non-null  int64 
 3   marital_status   30162 non-null  object
 4   occupation       30162 non-null  object
 5   relationship     30162 non-null  object
 6   race             30162 non-null  object
 7   sex              30162 non-null  object
 8   capital_gain     30162 non-null  int64 
 9   capital_loss     30162 non-null  int64 
 10  hours_per_week   30162 non-null  int64 
 11  native_country   30162 non-null  object
 12  capital_gain_yn  30162 non-null  int64 
 13  capital_loss_yn  30162 non-null  int64 
dtypes: int64(7), object(7)
memory usage: 3.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15060 entries, 0 to 15059
Data columns (total 14 columns):
 #   Co

In [44]:
train_df = pd.concat([X_train, y_train], axis = 1)

for _col in COL_NUM :
  print('-'*70)
  print(_col)
  print(train_df.groupby(COL_Y)[_col].describe(), end='\n\n')

----------------------------------------------------------------------
age
          count      mean        std   min   25%   50%   75%   max
target                                                            
0       22654.0  36.60806  13.464631  17.0  26.0  34.0  45.0  90.0
1        7508.0  43.95911  10.269633  19.0  36.0  43.0  51.0  90.0

----------------------------------------------------------------------
education_num
          count       mean       std  min   25%   50%   75%   max
target                                                           
0       22654.0   9.629116  2.413596  1.0   9.0   9.0  10.0  16.0
1        7508.0  11.606420  2.368423  2.0  10.0  12.0  13.0  16.0

----------------------------------------------------------------------
hours_per_week
          count       mean        std  min   25%   50%   75%   max
target                                                            
0       22654.0  39.348592  11.950774  1.0  38.0  40.0  40.0  99.0
1        7508.0  45

age, education_num, hours_per_week 모두 종속변수(target)값이 1일 때의 평균이 0일 때의 평균보다 더 크다.

In [45]:
for _col in COL_CAT :
  print('-'*70)
  print(_col)
  print(train_df.groupby(_col, as_index=False)[COL_Y].mean().sort_values(by=COL_Y, ascending=False), end='\n\n')

----------------------------------------------------------------------
workclass
          workclass    target
3      Self-emp-inc  0.558659
0       Federal-gov  0.387063
1         Local-gov  0.294630
4  Self-emp-not-inc  0.285714
5         State-gov  0.268960
2           Private  0.218792
6       Without-pay  0.000000

----------------------------------------------------------------------
marital_status
          marital_status    target
1      Married-AF-spouse  0.476190
2     Married-civ-spouse  0.454959
0               Divorced  0.107262
6                Widowed  0.096735
3  Married-spouse-absent  0.083784
5              Separated  0.070288
4          Never-married  0.048324

----------------------------------------------------------------------
occupation
           occupation    target
3     Exec-managerial  0.485220
9      Prof-specialty  0.448489
10    Protective-serv  0.326087
12       Tech-support  0.304825
11              Sales  0.270647
2        Craft-repair  0.225310
13   

학습 데이터에서 종속변수가 1인 비율은 약 0.249이다. 위 결과를 보면 범주값별로 종속변수 평균이 다양하게 나타나서 종속변수 예측에 사용할 만하다고 볼 수 있다.
참고로 native country 변수의 경우 41개의 고유한 값을 갖는다. 이렇게 한 변수가 가질 수 있는 고유한 값의 개수를 카디널리티(Cardinality)라는 말로 표현한다. 카디널리티가 높은 변수는 모델 성능에 악영향을 준다. 보통 100개 이상의 고유한 값을 가지면 카디널리티가 높다고 하는데 이 경우 해당 변수를 제거하거나, 일부 범주를 합쳐 카디널리티를 낮출 수 있다. 또는 다루기 까다롭지만 타겟 인코딩을 적용할 수 있다. 우리는 학습 데이터 양이 적지 않고, 변수 값들을 봤을 때 카디널리티에 의한 모델 성능 저하를 크게 우려하지 않아도 될 것으로 보인다.

**[레이블 인코딩]**

In [46]:
from sklearn.preprocessing import LabelEncoder
X = pd.concat([X_train, X_test])
for _col in COL_CAT :
  le = LabelEncoder()
  le.fit(X_train[_col])
  X_train[_col] = le.transform(X_train[_col])
  X_test[_col] = le.transform(X_test[_col])

레이블 인코딩을 한 데이터는 선형 모델에는 적합하지 않다. 레이블 인코딩한 변수 값이 의미를 가지는 것이 아니기 때문이다. 따라서 이어지는 데이터 모형 구축시에는 비선형 모델인 랜덤포레스트와 XGBoost 알고리즘을 사용하도록 한다.

**(4) 데이터 모형 구축**

In [47]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.3, stratify = y_train)

In [48]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_tr[COL_NUM] = scaler.fit_transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.fit_transform(X_val[COL_NUM])
X_test[COL_NUM] = scaler.fit_transform(X_test[COL_NUM])

Random Forest Model

In [49]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()
model_rf.fit(X_tr, y_tr.values.ravel()) # values.ravel() : 1차원 배열로 변환

RandomForestClassifier()

In [ ]:
# pip install --upgrade xgboost

In [50]:
from xgboost import XGBClassifier

model_xgb1 = XGBClassifier()
model_xgb1.fit(X_tr, y_tr.values.ravel())

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

XGBoost의 경우 eval_set을 사용하여 과적합을 방지할 수 있다. ⏬ ✈

In [51]:
# 모델 생성
model_xgb2 = XGBClassifier(n_estimators=1000, learning_rate=0.1, max_depth=10, eval_metric='auc') # eval_metric을 생성자에 추가

# 초기 모델 학습 (early_stopping_rounds 없이)
model_xgb2.fit(
    X_train,
    y_train.values.ravel(),
    eval_set=[(X_val, y_val)],
    verbose=10
)

# 조기 종료 구현
evals_result = model_xgb2.evals_result()
best_iteration = np.argmax(evals_result['validation_0']['auc'])

# 최적의 트리 개수로 재학습
model_xgb2 = XGBClassifier(n_estimators=best_iteration, learning_rate=0.1, max_depth=10, eval_metric='auc') # eval_metric을 생성자에 추가
model_xgb2.fit(X_tr, y_tr.values.ravel())

[0]	validation_0-auc:0.75475
[10]	validation_0-auc:0.74856
[20]	validation_0-auc:0.78447
[30]	validation_0-auc:0.71802
[40]	validation_0-auc:0.70218
[50]	validation_0-auc:0.68291
[60]	validation_0-auc:0.69984
[70]	validation_0-auc:0.69824
[80]	validation_0-auc:0.66552
[90]	validation_0-auc:0.66734
[100]	validation_0-auc:0.70599
[110]	validation_0-auc:0.69844
[120]	validation_0-auc:0.70248
[130]	validation_0-auc:0.71862
[140]	validation_0-auc:0.72608
[150]	validation_0-auc:0.73761
[160]	validation_0-auc:0.73174
[170]	validation_0-auc:0.72354
[180]	validation_0-auc:0.72656
[190]	validation_0-auc:0.70616
[200]	validation_0-auc:0.72864
[210]	validation_0-auc:0.73514
[220]	validation_0-auc:0.73896
[230]	validation_0-auc:0.74208
[240]	validation_0-auc:0.73779
[250]	validation_0-auc:0.73393
[260]	validation_0-auc:0.73668
[270]	validation_0-auc:0.73884
[280]	validation_0-auc:0.74609
[290]	validation_0-auc:0.72887
[300]	validation_0-auc:0.72025
[310]	validation_0-auc:0.73124
[320]	validation_0-

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=21, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

ROC-AUC 값 산출

In [52]:
from sklearn.metrics import roc_auc_score

y_pred_rf = model_rf.predict_proba(X_val)
y_pred_xgb1 = model_xgb1.predict_proba(X_val)
y_pred_xgb2 = model_xgb2.predict_proba(X_val)

score_rf = roc_auc_score(y_val, y_pred_rf[:, 1])
score_xgb1 = roc_auc_score(y_val, y_pred_xgb1[:, 1])
score_xgb2 = roc_auc_score(y_val, y_pred_xgb2[:, 1])

print('RF : ', score_rf)
print('XGB1 : ', score_xgb1)
print('XGB2 : ', score_xgb2)

RF :  0.8918650049611794
XGB1 :  0.9036399338753305
XGB2 :  0.896044769254851


변수별 중요도 확인 : 각 모델의 feature_importances 값이 변수 중요도를 나타낸다.  

In [53]:
pd.DataFrame({'feature' : X_tr.columns, 'fi_rf' : model_rf.feature_importances_,
              'fi_xgb1' : model_xgb1.feature_importances_,

              'fi_xgb2' : model_xgb2.feature_importances_})

,feature,fi_rf,fi_xgb1,fi_xgb2
0,age,0.214165,0.029772,0.012744
1,workclass,0.049690,0.021679,0.009372
2,education_num,0.137950,0.100126,0.080552
3,marital_status,0.074188,0.163584,0.248967
4,occupation,0.087846,0.028612,0.010492
5,relationship,0.112725,0.346064,0.422237
6,race,0.015916,0.017786,0.006794
7,sex,0.013608,0.032771,0.007453
8,capital_gain,0.104593,0.153233,0.142243
9,capital_loss,0.033100,0.061888,0.036422


변수 중요도에서 capital_gain_yn, capital_loss_yn은 각각 capital_gain, capital_loss보다 중요하지 않게 산출되므로 제외한다.

In [54]:
COL_DEL = ['capital_gain_yn', 'capital_loss_yn']

X_tr = X_tr.drop(COL_DEL, axis = 1)
X_val = X_val.drop(COL_DEL, axis = 1)
X_test = X_test.drop(COL_DEL, axis = 1)

GridSearchCV 함수를 이용해 하이퍼파라미터 조합을 시도해 볼 수 있다. GridSearchCV 인자 중 estimator는 튜닝에 사용할 기본 모델을, params_grid는 하이퍼파라미터 조합을, cv는 교차 검증시 몇 개 fold를 사용할 지를 의미한다.

> #경고 : GridSearchCV는 수행시간이 오래 걸린다! (11분 걸렸음!)

In [55]:
from sklearn.model_selection import GridSearchCV

grid_params = {
    'n_estimators' : [50, 100, 200],
    'max_depth' : [5, 10, 15],
    'min_samples_split' : [2, 5, 10],
    'min_samples_leaf' : [1, 2, 4]
    }

rf_cv = GridSearchCV(estimator = model_rf, param_grid = grid_params, cv = 5)
rf_cv.fit(X_tr, y_tr.values.ravel())

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 10, 15],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]})

In [ ]:
print(pd.DataFrame(rf_cv.cv_results_).head())

In [60]:
print('RF에서 가장 성능이 좋은 하이퍼파라미터 조합은 : ', rf_cv.best_params_)

RF에서 가장 성능이 좋은 하이퍼파라미터 조합은 :  {'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}


In [59]:
model_rf2 = RandomForestClassifier(n_estimators=200,
                                   max_depth=15,
                                   min_samples_leaf=1,
                                   min_samples_split=10)
model_rf2.fit(X_tr, y_tr.values.ravel())

y_pred_rf2 = model_rf2.predict_proba(X_val)
score_rf2 = roc_auc_score(y_val, y_pred_rf2[:, 1])
print('RF2 : ', score_rf2)

RF2 :  0.9145426059088341


In [61]:
grid_params = {
    'max_depth' : [3, 5, 7, 10],
    'min_child_weight' : [1, 2],
    'colsample_bytree' : [0.6, 0.8],
    'subsample' : [0.6, 0.8]
    }

xgb_cv = GridSearchCV(estimator = model_xgb1, param_grid = grid_params, cv = 5)
xgb_cv.fit(X_tr, y_tr.values.ravel())

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'colsample_bytree': [0.6, 0.8],
                         'max_depth': [3, 5, 7, 10], 'min_child_weight': [1, 2],
                         'subsample': [0.6, 0.8]})

In [62]:
print('XGB에서 가장 성능이 좋은 하이퍼파라미터 조합은 : ', xgb_cv.best_params_)

XGB에서 가장 성능이 좋은 하이퍼파라미터 조합은 :  {'colsample_bytree': 0.6, 'max_depth': 3, 'min_child_weight': 2, 'subsample': 0.8}


In [67]:
!pip install xgboost
import xgboost as xgb

params = {
    'colsample_bytree': 0.6,
    'max_depth': 3,
    'min_child_weight': 2,
    'subsample': 0.8
}

model_xgb3 = xgb.XGBClassifier(n_estimators=1000, learning_rate=0.05, eval_metric='auc', early_stopping_rounds=10) # Add early stopping
model_xgb3.set_params(**params)

# Use the 'eval_set' parameter to monitor performance on a validation set.
# Early stopping will be handled automatically based on the 'eval_metric'.
model_xgb3.fit(
    X_tr,
    y_tr,
    eval_set=[(X_val, y_val)],
    verbose=10
)

# Access the best score achieved during training
print(model_xgb3.best_iteration)
print(model_xgb3.best_score)

[0]	validation_0-auc:0.81709
[10]	validation_0-auc:0.88714
[17]	validation_0-auc:0.88585
7
0.8881027009878718


튜닝한 RF2 모델이 가장 높은 ROC-AUC 값을 산출하므로 해당 모델을 채택해 결과값을 내보낸다.

In [68]:
pred = model_rf2.predict_proba(X_test)[:, 1]
pd.DataFrame({'index' : X_test.index, 'target' : pred}).to_csv('census_y_test.csv', index = False)